In [16]:
import itertools
import json
import xarray as xr
import matplotlib.pyplot as plt
from LIM import utils as ut
from LSTM_enc_dec import *

plt.style.use("../plotting.mplstyle")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
data = xr.open_dataset("C:/Users/felix/PycharmProjects/ML_Climate_Science_Research_Project/LIM/data/ts_Amon_CESM2_piControl_r1i1p1f1.nc")["ts"]
#data = xr.open_dataset("./data/zos_Amon_CESM2_piControl_r1i1p1f1.nc")["zos"]
#data_old = xr.open_dataset("./data/ssta_1950_2021.nc")["ssta"]
mask = xr.open_dataset("C:/Users/felix/PycharmProjects/ML_Climate_Science_Research_Project/LIM/data/sftlf_fx_CESM2_historical_r1i1p1f1.nc")["sftlf"]

#14400 orginial size
data = data[:, :, :]

data = ut.apply_mask(mask, data)
#print("Data : {} + shape {}".format(data, data.shape))

data_anomalies = ut.calculate_monthly_anomalies(data)
#print("Month mean : {} + shape : {}".format(data_anomalies, data_anomalies.shape))

data_cropped =ut.crop_xarray(data_anomalies)
#print("Data cropped : {} + shape : {}".format(data_cropped, data_cropped.shape))


pca_10 = ut.SpatioTemporalPCA(data_cropped, n_components=20)
#pca_10 = ut.SpatioTemporalPCA(data_anomalies, n_components=20)
eof_10 = pca_10.eofs()
pc_10 = pca_10.principal_components()

Data cropped : <xarray.DataArray (lat: 64, lon: 49, time: 14400)>
array([[[ 2.02209473e-01,  1.17340088e-01,  4.94384766e-01, ...,
          8.75854492e-01,  6.95953369e-01,  1.23394775e+00],
        [ 2.11151123e-01,  1.29272461e-01,  4.34478760e-01, ...,
          8.53332520e-01,  6.09649658e-01,  1.13095093e+00],
        [ 2.19390869e-01,  1.32568359e-01,  3.57666016e-01, ...,
          8.30566406e-01,  5.58593750e-01,  1.03283691e+00],
        ...,
        [-1.58905029e-01, -2.84118652e-02,  8.29162598e-02, ...,
          4.74670410e-01,  9.83276367e-02,  1.40136719e-01],
        [-7.95898438e-02, -9.41680908e-01, -1.06283569e+00, ...,
         -4.00665283e-01, -3.01940918e-01,  3.83422852e-01],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan]],

       [[ 1.75354004e-01,  1.05651855e-01,  3.49700928e-01, ...,
          1.02838135e+00,  7.58544922e-01,  1.18096924e+00],
        [ 1.99768066e-01,  1.15692139

In [ ]:
from LSTM_model import *
#
# # Set random seed for reproducibility
# torch.manual_seed(42)
#
def hyperparameter_training_loop(data, hyperparams):

    # Generate all possible combinations of hyperparameters
    combinations = list(itertools.product(*hyperparams.values()))

    overall_best_model = {"hidden_size": None,
                         "learning_rate": None,
                         "num_epochs": None,
                         "loss": float('inf')}

    # Iterate over each hyperparameter combination
    for i, params in enumerate(combinations):
        print(f"Testing parameter combination {i+1}/{len(combinations)}: {params}")

        # Create a new model with the current hyperparameters
        hidden_size = params[0]
        learning_rate = params[1]
        num_epochs = params[2]
        num_layers = params[3]
        sequence_length = params[4]
        batch_size = params[5]

        dataloader = create_dataloader(data, sequence_length=sequence_length, batch_size=batch_size, shuffle=False)

        model = LSTMNetwork(1, hidden_size, num_layers)
        losses = train(model, dataloader, num_epochs, learning_rate)

        # Save the model and hyperparameters to a file
        result = {
            'hyperparameters': {
                'hidden_size': hidden_size,
                'learning_rate': learning_rate,
                'num_epochs': num_epochs,
                'num_layers': num_layers,
                'sequence_length': sequence_length,
                'best_loss': losses[-1],
                "losses": losses
            }
        }
        if losses[-1] < overall_best_model["loss"]:
            overall_best_model["hidden_size"] = params[0]
            overall_best_model["learning_rate"] = params[1]
            overall_best_model["num_epochs"] = params[2]
            overall_best_model["loss"] = losses[-1]
            overall_best_model["losses"] = losses

        filename = f"./model_trained_lstm/fnn_model_{i+1}.pt"
        with open(filename, 'w') as f:
            json.dump(result, f)

        print(f"Saved model and hyperparameters to {filename}\n")

    return overall_best_model
#
#
# # Create the DataLoader for first principal component
# data = np.array(data)[0]
# print("Data : {} + shape: {} + type: {}".format(data, data.shape, type(data)))
#
# # Hyperparameter search space
# hyperparams = {
#     'hidden_size': [32],
#     'learning_rate': [0.001, 0.002],
#     'num_epochs': [1000],
#     'num_layers': [3],
#     'sequence_length': [5],
#     'batch_size': [1]
# }
#
# #Train the model
# #train(model, dataloader, num_epochs, learning_rate)
# best_model = hyperparameter_training_loop(data, hyperparams)
#
# print(f"Best model: {best_model}")
# plot_loss_evolution(best_model["losses"], np.arange(0,best_model["num_epochs"], 100, dtype=int), best_model["learning_rate"], best_model["hidden_size"])

In [ ]:
# Data preparation for LSTM

# Set random seed for reproducibility
torch.manual_seed(42)

# Create the DataLoader for first principal component
data = pca_10.principal_components()
data = np.array(data)
#print("Data : {} + shape: {} + type: {}".format(data, data.shape, type(data)))

data = torch.from_numpy(data).type(torch.Tensor)

# Reshape the data if necessary (assuming a 2D tensor)
if len(data.shape) == 1:
    data = data.unsqueeze(1)

# Calculate the mean and standard deviation along the feature dimension
mean = torch.mean(data, dim=1, keepdim=True)
std = torch.std(data, dim=1, keepdim=True)

# Apply normalization using the mean and standard deviation
data = (data - mean) / std

index_train = int(0.8 * len(data[0, :]))
data_train = data[:, :index_train]
data_test = data[:, index_train:]

input_data, target_data = windowed_dataset(data_train, input_window=5, output_window=12)
input_data_test, target_data_test = windowed_dataset(data_test, input_window=5, output_window=12)

#print("Input data : {} + shape: {} + type: {}".format(input_data, input_data.shape, type(input_data)))



# convert windowed data from np.array to PyTorch tensor
X_train, Y_train, X_test, Y_test = numpy_to_torch(input_data, target_data, input_data_test, target_data_test)

print("X_train : {} + shape: {} + type: {}".format(X_train, X_train.shape, type(X_train)))
print("X_test : {} + shape: {} + type: {}".format(X_test, X_test.shape, type(X_test)))
print(X_train.shape[2])

In [ ]:
print("X-train shape : ", X_train.shape)
print("Y-train shape : ", Y_train.shape)
# specify model parameters and train
model = LSTM_seq2seq(input_size = X_train.shape[2], hidden_size = 32)
loss = model.train_model(X_train, Y_train, n_epochs = 200, target_len = 12, batch_size = 2, training_prediction = 'mixed-teacher-forcing', teacher_forcing_ratio = 0.6, learning_rate = 0.01, dynamic_tf = True)
#print("Loss : {} + shape: {} + type: {}".format(loss, loss.shape, type(loss)))